In [109]:
import re
import unidecode
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import random

In [60]:
#cleaning txt and creating list of words
file = open("liste_francais.txt","r",encoding="iso-8859-1")

words = [[] for x in range (30)]
for line in file:
    word = unidecode.unidecode(line).lower()
    word = word.replace('\n','')
    words[len(word)].append(word)
#print(words)


In [61]:
def create_pattern(template):
    new_template = ""
    for letter in template:
        if (letter != "."):
            new_template += "[^" + letter + "]"
        else:
            new_template += letter
    pattern = re.compile(new_template)
    return pattern

In [62]:
#first fonction return all words that have the correct letter at the right place
def rule_1 (list_of_words, template):
    #template = template.lower()
    pattern = re.compile(template)
    return [word for word in list_of_words if pattern.search(word) != None]

In [63]:
def yield_func_2(list_of_words,pattern,dictio):
    for word in list_of_words:
        bad_word = False
        if (pattern.search(word) != None):
            for k,v in dictio.items():
                #print (k,v,word.count(k))
                if (word.count(k) < v):
                    bad_word = True
                    break
            if (not bad_word):
                yield word 

In [73]:
#second fonction return all ords that obey the following rule
#good letter mean the letter is s in the word for at least the number of apparition in good good_letter + perfect_letter
#also mean it is currently at the wrong places 
def rule_2 (list_of_words, template,dictio):
    for letter in template:
        if (letter in dictio and letter != "."):
            dictio[letter] += 1
        elif (letter != "."):
            dictio[letter] = 1    
    pattern = create_pattern(template)      

    return list(yield_func_2(list_of_words,pattern,dictio))

In [65]:
def yield_func_3(list_of_words, pattern,dictio):
    for word in list_of_words:
        bad_word = False
        if (pattern.search(word) != None):
            for k,v in dictio.items():
                #print (k,v,word.count(k))
                if (word.count(k) > v):
                    bad_word = True
                    break
            if (not bad_word):
                yield word

In [66]:
#third fonction return all words that obey the following rule
#bad letter mean the letter is in the word for exactly the number of apparition in good_letter + perfect_letter
#can be 0
def rule_3 (list_of_words, template,dictio):
    for letter in template:
        if (not letter in dictio and letter != "."):
            dictio[letter] = 0     
    pattern = create_pattern(template)
    return list(yield_func_3(list_of_words,pattern, dictio))


In [67]:
#function that we need in the loop to solve the motus
def solve_motus(perfect_letter, good_letter, bad_letter, list_of_words):
    dictio = {}
    for letter in perfect_letter:
        if (letter in dictio and letter != "."):
            dictio[letter] += 1
        elif (letter != "."):
            dictio[letter] = 1
    if (len(perfect_letter) == len(good_letter) == len(bad_letter)):
        new_list = rule_1(list_of_words, perfect_letter)
        new_list = rule_2(new_list,good_letter,dictio)
        new_list = rule_3(new_list,bad_letter,dictio)
        return new_list

    else:
        print("input not same length")
        return False

In [150]:
#function that select a random answer in the correct list and write it on the website
def send_proposition(list_of_words,driver):
    #proposition = list_of_words[random.randint(0, len(list_of_words)-1)]
    proposition = list_of_words[0]
    for word in list_of_words[1:]:
        if (len(set(word)) > len(set(proposition))):
            proposition = word   
    written_letter = 1
    for letter in proposition[1:]:
        time.sleep(1)
        key = driver.find_element(By.CSS_SELECTOR, "[data-lettre=" + letter.upper() + "]")
        if (key):
            key.click()
            written_letter +=1
    if written_letter == len_word:
        key = driver.find_element(By.CSS_SELECTOR, "[data-lettre=_entree]")
        if (key):
            key.click()
    else:
        print("error during writting process")
    time.sleep(4)

In [151]:
#main loop range max is always 6 based on the website template
def do_loop(driver,list_of_words,len_word):
    for x in range (1,6):
        number_perfect_letter = 0
        letter_cases = driver.find_elements(By.XPATH, "//*[@id='grille']/table/tr["+ str(x) + "]/td")
        good_letter = ""
        bad_letter = ""
        perfect_letter = ""
        list_of_words = words[len(letter_cases)]
        for letter in letter_cases:
            #print(letter.get_attribute('class'))
            if (letter.get_attribute('class') == "bien-place resultat"):
                good_letter += "."
                bad_letter += "."
                perfect_letter += letter.get_attribute('innerHTML').lower()
                number_perfect_letter += 1
            elif (letter.get_attribute('class') == "mal-place resultat"):
                good_letter += letter.get_attribute('innerHTML').lower()
                bad_letter += "."
                perfect_letter += "."
            elif(letter.get_attribute('class') == "non-trouve resultat"):
                good_letter += "."
                bad_letter += letter.get_attribute('innerHTML').lower()
                perfect_letter += "."
        #print(number_perfect_letter,len_word)
        if(number_perfect_letter == len_word):
            print("win")
            break


        list_of_words = solve_motus(perfect_letter, good_letter, bad_letter, list_of_words)
        print(list_of_words)

        if (list_of_words== []):
            print("no word found")
            break

        send_proposition(list_of_words,driver)

In [152]:
#selenium set up + closing pop up page on target site
chrome_options = webdriver.ChromeOptions()
#incognito allow multiple execution of the script
chrome_options.add_argument("--incognito")
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s,chrome_options=chrome_options)
driver.maximize_window()
driver.get('https://sutom.nocle.fr/')
tips_windows = driver.find_element(By.ID, "panel-fenetre-bouton-fermeture-icone")
if (tips_windows):
    tips_windows.click()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/home/jeremie/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache
/tmp/ipykernel_7251/3760430506.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s,chrome_options=chrome_options)


In [153]:
#first proposition
letter_cases = driver.find_elements(By.XPATH, "//*[@id='grille']/table/tr[1]/td")
len_word = len(letter_cases)
list_of_words = words[len_word]
good_letter = ""
bad_letter = ""
perfect_letter = ""
for letter in letter_cases:
    perfect_letter += letter.get_attribute('innerHTML').lower()
    bad_letter += "."
    good_letter += "."
list_of_words = solve_motus(perfect_letter, good_letter, bad_letter, list_of_words)
send_proposition(list_of_words,driver)

do_loop(driver,list_of_words,len_word)

a
c
h
e
r
['gerant', 'girafe', 'graine', 'gramme', 'grande', 'grappe', 'grasse']
e
r
a
n
t
['girafe']
i
r
a
f
e
win
